In [1]:
import pandas as pd
import warnings
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio
import mapboxgl
from mapboxgl.viz import *
from mapboxgl.utils import df_to_geojson
import matplotlib.pyplot as plt
from mapboxgl.utils import create_color_stops
from mapboxgl.utils import create_radius_stops

warnings.filterwarnings(action='ignore')
pio.renderers.default = 'iframe'

# 데이터 로드
- 일반대학 / 전문대학 포함: '분류' 컬럼에서 일반대학은 university로, 전문대학은 junior college로 구분함.
- 신입생 충원율 = 입학자_정원내 / 모집인원_정원내
- 입학자_전체_계 = 정원내 및 정원외 입학자 포함
- 정원내 모집인원은 정원 이월 등으로 입학정원보다 많을 수 있음.

In [2]:
df = pd.read_csv('data/2020년_대학_통계.csv')
df_major = pd.read_csv('data/2020년_대학_학과별_통계.csv')

In [3]:
df = df[df['학교상태']!='폐교']
df_major = df_major[df_major['상태']!='폐교']

# 지역별 / 학제별 대학 수 ( 일반대학 및 전문대학 )

In [4]:
#피벗 테이블 생성
univs_count = pd.pivot_table(df, values='학교명', index=['시도'], columns=['분류'], aggfunc='count').fillna(np.nan).sort_values(by=['시도'], ascending=False)
univs_count =univs_count.round(2)

def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

colorscale = [[0, 'navy'], [1, 'plum']]
font_colors = ['white', 'black']

# 결측값 마스킹
mask = np.zeros_like(univs_count, dtype=bool)
df_mask = univs_count.mask(mask)

dict_count = df_to_plotly(df_mask)

#히트맵 생성
fig = ff.create_annotated_heatmap(dict_count['z'], y=dict_count['y'], x = dict_count['x'],
                                  colorscale='Temps',
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )
fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='지역/학제별 학교수', 
    title_x=0.5, 
    width=400, 
    height=500,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.write_image("images/지역_학제별_학교수.png")
fig.show()

# 지역 / 학제별 신입생 충원율 분석 ( 일반대학 및 전문대학 )

In [5]:
fig = px.histogram(df, x="신입생_충원율", color='분류', title='대학별 신입생 충원율 히스토그램', opacity=0.8,  marginal="box", hover_data=df.columns)

fig.update_layout(
    title_text='대학별 신입생 충원율 히스토그램', 
    title_x=0.5, 
    width=800, 
    height=500,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30
)
fig.write_image("images/대학별_신입생_충원율_히스토그램.png")
fig.show()

In [6]:
freshman_rate_pv = pd.pivot_table(df, values='신입생_충원율', index=['시도'], columns=['분류'], aggfunc=np.mean).fillna(np.nan).sort_values(by=['시도'], ascending=False)
freshman_rate_pv =freshman_rate_pv.round(2)

colorscale = [[0, 'navy'], [1, 'plum']]
font_colors = ['white', 'black']

mask = np.zeros_like(freshman_rate_pv, dtype=bool)
df_mask = freshman_rate_pv.mask(mask)
dict_count = df_to_plotly(df_mask)

fig = ff.create_annotated_heatmap(dict_count['z'], y=dict_count['y'], x = dict_count['x'],
                                  colorscale='Tealrose',
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='지역/학제별 신입생 충원율', 
    title_x=0.5, 
    width=400, 
    height=500,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.write_image("images/지역_학제별_신입생_충원율.png")
fig.show()

# 지역 / 설립유형별 신입생 충원율 분석 ( 일반대학 및 전문대학 )

In [7]:
freshman_rate_pv_2 = pd.pivot_table(df, values='신입생_충원율', index=['시도'], columns=['설립'], aggfunc=np.mean).fillna(np.nan).sort_values(by=['시도'], ascending=False)
freshman_rate_pv_2 =freshman_rate_pv_2.round(2)

colorscale = [[0, 'navy'], [1, 'plum']]
font_colors = ['white', 'black']

mask = np.zeros_like(freshman_rate_pv_2, dtype=bool)
df_mask = freshman_rate_pv_2.mask(mask)

dict_count = df_to_plotly(df_mask)

fig = ff.create_annotated_heatmap(dict_count['z'], y=dict_count['y'], x = dict_count['x'],
                                  colorscale='Pinkyl',
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='지역/설립유형별 신입생 충원율', 
    title_x=0.5, 
    width=550, 
    height=500,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30,
    template = 'plotly_white'
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.write_image("images/지역_설립유형별_신입생_충원율.png")
fig.show()

# 지역 / 설립유형별 신입생 충원율 분석 ( 일반대학 )

In [8]:
df_univ = df[df['분류']=='university']
freshman_rate_pv_univ = pd.pivot_table(df_univ, values='신입생_충원율', index=['시도'], columns=['설립'], aggfunc=np.mean).fillna(np.nan).sort_values(by=['시도'], ascending=False)
freshman_rate_pv_univ =freshman_rate_pv_univ.round(2)

colorscale = [[0, 'navy'], [1, 'plum']]
font_colors = ['white', 'black']

mask = np.zeros_like(freshman_rate_pv_univ, dtype=bool)
df_mask = freshman_rate_pv_univ.mask(mask)

dict_count = df_to_plotly(df_mask)

fig = ff.create_annotated_heatmap(dict_count['z'], y=dict_count['y'], x = dict_count['x'],
                                  colorscale='Bluyl',
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='지역 / 설립유형별 신입생 충원율 (일반대학)', 
    title_x=0.5, 
    width=550, 
    height=500,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30,
    template = 'plotly_white'
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.write_image("images/지역_설립유형별_신입생_충원율(일반대학).png")
fig.show()

# 지역 / 설립유형별 신입생 충원율 분석 ( 전문대학 )

In [9]:
df_col = df[df['분류']=='junior_college']
freshman_rate_pv_col = pd.pivot_table(df_col, values='신입생_충원율', index=['시도'], columns=['설립'], aggfunc=np.mean).fillna(np.nan).sort_values(by=['시도'], ascending=False)
freshman_rate_pv_col =freshman_rate_pv_col.round(2)

colorscale = [[0, 'navy'], [1, 'plum']]
font_colors = ['white', 'black']

mask = np.zeros_like(freshman_rate_pv_col, dtype=bool)
df_mask = freshman_rate_pv_col.mask(mask)

dict_count = df_to_plotly(df_mask)

fig = ff.create_annotated_heatmap(dict_count['z'], y=dict_count['y'], x = dict_count['x'],
                                  colorscale='Tealgrn',
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='지역 / 설립유형별 신입생 충원율 (전문대학)', 
    title_x=0.5, 
    width=400, 
    height=500,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30,
    template = 'plotly_white'
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

        
fig.write_image("images/지역_설립유형별_신입생_충원율(전문대학).png")
fig.show()

# 신입생 충원율 지도 시각화

In [11]:
geo_df = df_to_geojson(df=df, lat='위도', lon='경도')
center = [127.77, 35.91] 
token = '개인토큰입력'

viz = CircleViz(
    geo_df,
    access_token=token,
    radius=3,
    center=center,
    zoom=6.3,
    width='700px',
    height='700px',
    style='mapbox://styles/mapbox/dark-v10')

viz.color_property = '신입생_충원율'
viz.color_stops = create_color_stops([80,85, 90, 95, 100], colors='RdBu')

viz.show()

# 학과별 분석

In [12]:
pd.pivot_table(df_major, values='정원내_신입생_충원율(정원내 입학자 / 정원내 모집인원)', index=['대계열'], columns=['분류'], aggfunc=np.mean).fillna(0).sort_values(by=['대계열'], ascending=False)

분류,junior_college,university
대계열,,
자연계열,71.121379,94.935476
인문계열,70.507407,78.911731
의약계열,67.759680,94.324096
예체능계열,70.747947,90.268535
사회계열,63.337449,86.482249
교육계열,57.112217,96.471363
공학계열,59.394291,90.149773


In [13]:
major_pv = pd.pivot_table(df_major, values='정원내_신입생_충원율(정원내 입학자 / 정원내 모집인원)', index=['대계열'], columns=['분류'], aggfunc=np.mean).fillna(0).sort_values(by=['대계열'], ascending=False)
major_pv =major_pv.round(2)

colorscale = [[0, 'navy'], [1, 'plum']]
font_colors = ['white', 'black']

mask = np.zeros_like(major_pv, dtype=bool)
df_mask = major_pv.mask(mask)

dict_count = df_to_plotly(df_mask)

fig = ff.create_annotated_heatmap(dict_count['z'], y=dict_count['y'], x = dict_count['x'],
                                  colorscale=px.colors.diverging.RdBu,
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='학과계열/학제별 신입생 충원율', 
    title_x=0.5, 
    width=400, 
    height=500,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.write_image("images/학과계열_학제별_신입생_충원율.png")
fig.show()

In [14]:
major_pv = pd.pivot_table(df_major, values='정원내_신입생_충원율(정원내 입학자 / 정원내 모집인원)', index=['중계열'],
                    columns=['분류'], aggfunc=np.mean).fillna(0).sort_values(by=['중계열'], ascending=False)
major_pv =major_pv.round(2)

mask = np.zeros_like(major_pv, dtype=bool)
df_mask = major_pv.mask(mask)

dict_count = df_to_plotly(df_mask)

fig = ff.create_annotated_heatmap(dict_count['z'], y=dict_count['y'], x = dict_count['x'],
                                  colorscale='Purpor',
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1, 
                                 )

fig.update_xaxes(side="bottom")


fig.update_layout(
    title_text='학과 중계열/학제별 신입생 충원율', 
    title_x=0.5, 
    width=400, 
    height=1000,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.write_image("images/학과_중계열_학제별_신입생_충원율.png")
fig.show()

In [15]:
inmun_major = df_major[df_major['중계열']=='인문과학']
major_pv = pd.pivot_table(inmun_major, values='정원내_신입생_충원율(정원내 입학자 / 정원내 모집인원)', index=['소계열'],
                    columns=['설립구분'], aggfunc=np.mean).sort_values(by=['소계열'], ascending=False)
major_pv =major_pv.round(2)

mask = np.zeros_like(major_pv, dtype=bool)
df_mask = major_pv.mask(mask)

dict_count = df_to_plotly(df_mask)

fig = ff.create_annotated_heatmap(dict_count['z'], y=dict_count['y'], x = dict_count['x'],
                                  colorscale='Purpor',
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1, 
                                 )

fig.update_xaxes(side="bottom")


fig.update_layout(
    title_text='인문과학 계열 신입생 충원율', 
    title_x=0.5, 
    width=700, 
    height=500,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30
)

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.write_image("images/인문과학계열_신입생_충원율.png")
fig.show()

In [16]:
#교양인문학 학과 목록 확인
list(set(inmun_major[inmun_major['소계열']=='교양인문학']['학과(모집단위)']))

['인문계열',
 '디아코니아학부',
 '인문학부',
 '전공자유학부',
 '글로벌자유전공학과',
 '문화창의학부',
 '연계전공학부',
 '자유전공학부',
 '인문자율전공학부',
 '자율전공학부(인문사회계열)',
 '창의융합학부(인문사회계열)',
 '호크마교양대학',
 '자율전공학부(4년)',
 '인문계',
 '인문예술・사회계열',
 '스포츠산업학과',
 '융합인재학부',
 '크리에이티브인문학부',
 '새내기과정학부(인문사회계열)',
 '인문사회융합계열',
 '창의융합인재학부',
 '인문・사회과학전공',
 '융합전공학부(인문)',
 '인문과학대학',
 '스크랜튼학부',
 '서울캠퍼스 자율전공',
 '자율전공부 인문사회자율전공계열',
 '인문문화학부',
 '문화교양학과',
 '인문사회계열',
 '휴먼학부',
 '글로벌융합대학',
 '스포츠아웃도어학과',
 '기초교육학부',
 '역사관광·외교학부',
 '후마니타스학과',
 '언더우드학부',
 '영미인문학과',
 '글로벌자율전공학부',
 '인문융합자율학부',
 '쿰칼리지',
 '인문과학부',
 '자유전공학부(인문계열)',
 '자유전공학부(충주/증평)',
 '자유전공',
 '융합인문사회과학부',
 '글로벌엘리트학부',
 '창의융합교육원(인문사회)',
 '글로벌자율전공학부(이학계열)',
 '자유전공학부(의왕)',
 '인문문화융합학부',
 '융합사회학부',
 '기타모집단위',
 '파이데이아창의인재학과',
 '자율전공학부(1년)',
 '자율전공학부(인문・자연)',
 '인문과학계열']

# 2018 - 2020 대학별 신입생 충원율 증감 분석

In [17]:
df_by_years = pd.read_csv('data/3개년_대학_신입생_충원율.csv')

In [18]:
df_2019 =pd.read_excel('data/대학_신입생_충원_현황_2019.xlsx')
df_2018 =pd.read_excel('data/대학_신입생_충원_현황_2018.xlsx')
df_2019['입학자_정원내'] = df_2019['입학자_정원내_남'] + df_2019['입학자_정원내_여'] 
df_2018['입학자_정원내'] = df_2018['입학자_정원내_남'] + df_2018['입학자_정원내_여'] 

In [19]:
df_by_years['2019_to_2020_증감율'] = df_by_years['2020'] - df_by_years['2019']
df_by_years['2018_to_2019_증감율'] = df_by_years['2019'] - df_by_years['2018']
df_by_years['2018_to_2020_증감율'] = df_by_years['2020'] - df_by_years['2018']

In [20]:
df_re = df[['학교명', '모집인원_정원내', '입학자_정원내']]
df_by_years = pd.merge(df_re, df_by_years, how='left', on='학교명')

In [21]:
df_2018_to_2020_top20 = df_by_years.sort_values(by=['2018_to_2020_증감율'], ascending=False)[['학교명', '2018_to_2020_증감율']][:20]

In [22]:
top20_list = list(df_2018_to_2020_top20['학교명'])
top20_2018 = df_2018[df_2018['학교'].isin(top20_list)][['학교', '모집인원_정원내', '입학자_정원내']].rename(columns={'학교': '학교명', '모집인원_정원내':'모집인원_정원내_2018', '입학자_정원내':'입학자_정원내_2018'})
top20_2020 = df[df['학교명'].isin(top20_list)][['학교명', '모집인원_정원내', '입학자_정원내']].rename(columns={'모집인원_정원내':'모집인원_정원내_2020', '입학자_정원내':'입학자_정원내_2020'})
top20_merged = pd.merge(top20_2018, top20_2020, on='학교명', how='left')
top20_merged = pd.merge(df_2018_to_2020_top20, top20_merged, on='학교명', how='left')

In [23]:
top20_merged = top20_merged[['학교명', '2018_to_2020_증감율', '모집인원_정원내_2018','모집인원_정원내_2020',  '입학자_정원내_2018','입학자_정원내_2020']]
top20_merged

,학교명,2018_to_2020_증감율,모집인원_정원내_2018,모집인원_정원내_2020,입학자_정원내_2018,입학자_정원내_2020
0,인천가톨릭대학교,28.6,23,16.0,15,15.0
1,대전신학대학교,18.3,60,32.0,19,16.0
2,한국열린사이버대학교,13.5,1000,1000.0,865,1000.0
3,서울사이버대학교,13.1,3270,3270.0,2274,2702.0
4,대전가톨릭대학교,12.5,40,40.0,9,14.0
5,대구경북과학기술원,11.4,220,220.0,206,231.0
6,한려대학교,10.0,333,205.0,92,77.0
7,대구사이버대학교,8.6,1500,1500.0,1021,1150.0
8,칼빈대학교,8.6,93,92.0,85,92.0
9,부산디지털대학교,8.1,1000,1000.0,707,788.0


# 학령인구를 바탕으로 한 신입생 충원율 예측

In [24]:
pop_by_grade = pd.read_csv('연도별_지역별_학년별_인구.csv')
pop_by_grade = pop_by_grade.set_index('시도')

In [25]:
drop_pecentage_2019= (pop_by_grade['고2_2019'].loc['전체'] - pop_by_grade['고3_2020'].loc['전체']) / pop_by_grade['고2_2019'].loc['전체']
drop_pecentage_2018= (pop_by_grade['고1_2018'].loc['전체'] - pop_by_grade['고2_2019'].loc['전체']) / pop_by_grade['고1_2018'].loc['전체']
drop_pecentage_2017= (pop_by_grade['중3_2017'].loc['전체'] - pop_by_grade['고1_2018'].loc['전체']) / pop_by_grade['중3_2017'].loc['전체']
drop_pecentage_2016 =(pop_by_grade['중2_2016'].loc['전체'] - pop_by_grade['중3_2017'].loc['전체']) / pop_by_grade['중2_2016'].loc['전체']
drop_pecentage_2015 =(pop_by_grade['중1_2015'].loc['전체'] - pop_by_grade['중2_2016'].loc['전체']) / pop_by_grade['중1_2015'].loc['전체']
mean_drop_perc = np.mean([drop_pecentage_2015, drop_pecentage_2016, drop_pecentage_2017, drop_pecentage_2018, drop_pecentage_2015])

In [26]:
high_3rd_2020 = pop_by_grade['고3_2020'].loc['전체'] #2020년 고3 인원
exp_high_3rd_2021 = pop_by_grade['고2_2020'].loc['전체'] *  (1 - mean_drop_perc) #2021년 고3 학령인구 예측
exp_high_3rd_2022 = pop_by_grade['고1_2020'].loc['전체'] *  ((1 - mean_drop_perc)**2) #2022년 고3 학령인구 예측
exp_high_3rd_2023 = pop_by_grade['중3_2020'].loc['전체'] *  ((1 - mean_drop_perc)**3) #2023년 고3 학령인구 예측
exp_high_3rd_2024 = pop_by_grade['중2_2020'].loc['전체'] *  ((1 - mean_drop_perc) **4) #2024년 고3 학령인구 예측
exp_high_3rd_2025 = pop_by_grade['중1_2020'].loc['전체'] *  ((1 - mean_drop_perc) **5) #2025년 고3 학령인구 예측

In [27]:
#2020대비 2021~2025년 고3 학령인구 비율
print(np.round(exp_high_3rd_2021,2) / high_3rd_2020)
print(np.round(exp_high_3rd_2022,2) / high_3rd_2020)
print(np.round(exp_high_3rd_2023,2) / high_3rd_2020)
print(np.round(exp_high_3rd_2024,2) / high_3rd_2020)
print(np.round(exp_high_3rd_2025,2) / high_3rd_2020)

1.0237559082086998
1.0042345473227539
0.9200218061422537
0.9479474369220231
1.036523027742893


In [28]:
# 2022 - 2026 신입생 충원율 예측치
# 2020년 충원율 평균치에 2020년 대비 2021~2025년 고3 학령인구 인구 비율을 곱해 대략적으로 예상
expected_figure = []
expected_figure.append(np.mean(df['신입생_충원율']) * (np.round(exp_high_3rd_2021,2) / high_3rd_2020))
expected_figure.append(np.mean(df['신입생_충원율']) * (np.round(exp_high_3rd_2022,2) / high_3rd_2020))
expected_figure.append(np.mean(df['신입생_충원율']) * (np.round(exp_high_3rd_2023,2) / high_3rd_2020))
expected_figure.append(np.mean(df['신입생_충원율']) * (np.round(exp_high_3rd_2024,2) / high_3rd_2020))
expected_figure.append(np.mean(df['신입생_충원율']) * (np.round(exp_high_3rd_2025,2) / high_3rd_2020))

In [29]:
# 2022 - 2026 일반대학 신입생 충원율 예측치
df_univ = df[df['분류']=='university']
expected_figure_univs = []
expected_figure_univs.append(np.mean(df_univ['신입생_충원율']) * (np.round(exp_high_3rd_2021,2) / high_3rd_2020))
expected_figure_univs.append(np.mean(df_univ['신입생_충원율']) * (np.round(exp_high_3rd_2022,2) / high_3rd_2020))
expected_figure_univs.append(np.mean(df_univ['신입생_충원율']) * (np.round(exp_high_3rd_2023,2) / high_3rd_2020))
expected_figure_univs.append(np.mean(df_univ['신입생_충원율']) * (np.round(exp_high_3rd_2024,2) / high_3rd_2020))
expected_figure_univs.append(np.mean(df_univ['신입생_충원율']) * (np.round(exp_high_3rd_2025,2) / high_3rd_2020))

In [30]:
# 2022 - 2026 전문대 신입생 충원율 예측치
df_col = df[df['분류']=='junior_college']
expected_figure_cols = []
expected_figure_cols.append(np.mean(df_col['신입생_충원율']) * (np.round(exp_high_3rd_2021,2) / high_3rd_2020))
expected_figure_cols.append(np.mean(df_col['신입생_충원율']) * (np.round(exp_high_3rd_2022,2) / high_3rd_2020))
expected_figure_cols.append(np.mean(df_col['신입생_충원율']) * (np.round(exp_high_3rd_2023,2) / high_3rd_2020))
expected_figure_cols.append(np.mean(df_col['신입생_충원율']) * (np.round(exp_high_3rd_2024,2) / high_3rd_2020))
expected_figure_cols.append(np.mean(df_col['신입생_충원율']) * (np.round(exp_high_3rd_2025,2) / high_3rd_2020))

In [31]:
years = ['2022', '2023', '2024', '2025', '2026']
expected_fig_years = pd.DataFrame({'연도':years, '예측치':expected_figure, '예측치_일반대학': expected_figure_univs, '예측치_전문대학':expected_figure_cols})
fig = px.bar(expected_fig_years, y=['예측치_일반대학','예측치_전문대학'], x='연도', barmode='group')

texts = [expected_figure_univs, expected_figure_cols]
for i, t in enumerate(texts):
    fig.data[i].text = np.round(t,2)
    fig.data[i].textposition = 'outside'
    
fig.update_layout(
    title_text='2022-2026 신입생 충원율 예측', 
    title_x=0.2, 
    width=600, 
    height=400,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    margin_l=30,
    margin_r=30,
    margin_t=30,
    margin_b=30,
    uniformtext_minsize=8, 
    uniformtext_mode='hide', 
    yaxis=dict(range=[80,100]))


fig.write_image("images/연도별신입생_충원율_예측.png")
fig.show()